In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from tools.standard_name_utils import optimize_parameters
from tools.common_utils import highlight_diff, read_jsonl

model_path = "../ChatGLM2-6B/ptuning/output/checkpoint-100"
top_p=0.7 
temperature=0.95

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
model = model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 3/3 [02:09<00:00, 43.09s/it]


In [ ]:
response, history = model.chat(tokenizer, "你好", history=[], top_p=top_p, temperature=temperature)
print(response)

### 评估一下训练数据的情况

In [3]:
json_file_path = 'data/train.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

,input,output
0,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
1,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
2,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
3,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""tool_name""..."
4,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""tool_name""..."


In [5]:
from random import random

total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    if random() <= 0.9:
        continue
    
    input = row["input"]
    output = row["output"]
    response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
    
    total_eval_count += 1
    if response == output:
        correct_count += 1
    else:
        print("-----data index-----")
        print(index)
        print("-----query input-----")
        print(input)
        print("-----output diff-----")
        print(highlight_diff(output, response))
        print(response)
        print()
    
    if total_eval_count == 100:
        break
    
print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

-----data index-----
7
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：我今年5月1日用5000块买的国金金腾通C，如果我现在全部卖出，交易费用是多少 
 query中提到的产品标准名可能是：国金金腾通货币市场证券投资基金C类、华宝中证1000指数证券投资基金C类、鹏华中证1000指数增强型证券投资基金C类、长信中证1000指数增强型证券投资基金C类、财通中证1000指数增强型证券投资基金C类、浙商中证1000指数增强型证券投资基金C类、明亚中证1000指数增强型证券投资基金C类、招商中证1000指数增强型证券投资基金C类、富国中证1000优选股票型证券投资基金C类、太平中证1000指数增强型证券投资基金C类、天弘中证1000指数增强型证券投资基金C类、国金中证1000指数增强型证券投资基金C类、博道中证1000指数增强型证券投资基金C类、博时中证1000指数增强型证券投资基金C类、华安中证1000指数增强型证券投资基金C类、华夏中证1000指数增强型证券投资基金C类、兴银中证1000指数增强型证券投资基金C类、中银中证1000指数增强型证券投资基金C类、中欧中证1000指数增强型证券投资基金C类、汇添富中证1000指数增强型证券投资基金C类、建信中证1000交易型开放式指数证券投资基金、广发中证1000交易型开放式指数证券投资基金、富国中证1000交易型开放式指数证券投资基金、国联安中证1000指数增强型证券投资基金C类、南方中证1000交易型开放式指数证券投资基金、华夏中证1000交易型开放式指数证券投资基金、财通资管中证1000指数增强型证券投资基金C类、西部利得中证1000指数增强型证券投资基金C类、申万菱信中证1000指数增强型证券投资基金C类、汇添富中证1000交易型开放式指数证券投资基金、景顺长城中证1000指数增强型证券投资基金C类、易方达中证1000交易型开放式指数证券投资基金、工银瑞信中证1000指数增强型证券投资基金C类、国泰君安中证1000指数增强型证券投资基金C类、华泰柏瑞中证1000指数增强型证券投资基金C类、创金合信国证1000指数发起式证券投资基金C类、农银汇理中证1000指数增强型证券投资基金C类、中信建投中证

In [ ]:
json_file_path = 'data/dev.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

In [ ]:
data_stock = pd.read_excel('data/标准名.xlsx',sheet_name='股票标准名')
data_fund = pd.read_excel('data/标准名.xlsx',sheet_name='基金标准名')

fund_standard_name = data_fund['标准基金名称'].to_list()
stock_standard_name = data_stock['标准股票名称'].to_list()

In [ ]:
stock_standard_name[:5]

In [ ]:
from random import random

total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    if random() <= 0.9:
        continue
    
    input = row["input"]
    output = row["output"]
    response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
    
    optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
    
    total_eval_count += 1
    if optimized_resp == output:
        correct_count += 1
    else:
        print("-----data index-----")
        print(index)
        print("-----query input-----")
        print(input)
        print("-----output diff-----")
        print(highlight_diff(output, optimized_resp))
        print(optimized_resp)
        print()
    
    if total_eval_count == 100:
        break
    
print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

In [ ]:
test_input = df.loc[35]["input"]
test_input

In [ ]:
test_input = df.loc[19]["input"]
test_input += " 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价"
response, history = model.chat(tokenizer=tokenizer, query=test_input, history=[], top_p=top_p, temperature=temperature)
optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)

output = df.loc[19]["output"]
print(highlight_diff(output, optimized_resp))
print(optimized_resp)

### 输出新的提交文件

In [ ]:
test_json_file_path = 'data/test_a.json'
test_data_list = read_jsonl(test_json_file_path)

test_df = pd.DataFrame(test_data_list)
test_df.head()

In [ ]:
total_eval_count = 0

with open('data/submit.txt','w', encoding="utf-8") as n:
    for index, row in test_df.iterrows():    
        input = row["input"]
        response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
        
        optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
        
        total_eval_count += 1
        n.write(optimized_resp+'\n')
        if total_eval_count % 10 == 0:
            print("现在是第" + f"{total_eval_count}" + "条数据")